In [1]:
#Wenn man den typen bestimmte hat, sollte man zunächst alle Pflicht-Felder des Bibtex-Eintrages raussuchen und bestimmen.
#Dazu die Pflichtattribute in einer Schleife durchgehen.

In [2]:
import torch
import pandas as pd
torch.cuda.is_available()
from transformers import pipeline

True

@book{advanced_physics, author = {Stephen Hawking and Brian Greene and Maria Trump and Susan Soy}, title = {Advanced Concepts in Theoretical Physics}, publisher = {Cambridge University Press}, year = {2025}, edition = {3}, volume = {1}, series = {Advanced Studies in Physics}, address = {Cambridge, UK}, month = {May}, note = {A seminal work in the field of theoretical physics}, isbn = {978-0-521-76948-0}, doi = {10.1017/example.book.2025}, url = {https://www.cambridge.org/advanced_physics}, annote = {Widely cited in the physics community}, abstract = {This book explores cutting-edge theories and concepts in theoretical physics...}, keywords = {Theoretical Physics, Quantum Mechanics, String Theory}, language = {English}, price = {75.00}, size = {600 pages}, lccn = {2020934576}, mrnumber = {MR3070071} }

@article{quantum_entanglement,
  author        = {Albert Einstein and Boris Podolsky and Nathan Rosen},
  title         = {Can Quantum-Mechanical Description of Physical Reality Be Considered Complete?},
  journal       = {Physical Review},
  year          = {1935},
  volume        = {47},
  number        = {10},
  pages         = {777-780},
  month         = {May},
  note          = {EPR Paradox paper, fundamental for quantum mechanics},
  doi           = {10.1103/PhysRev.47.777},
  url           = {https://journals.aps.org/pr/abstract/10.1103/PhysRev.47.777},
  abstract      = {In this paper, the authors discuss the EPR paradox and challenge the completeness of quantum mechanics...},
  keywords      = {Quantum Mechanics, EPR Paradox, Physical Reality},
  language      = {English},
  publisher     = {American Physical Society}
}

@inproceedings{deep_learning,
  author        = {Geoffrey Hinton and Yoshua Bengio and Yann LeCun},
  title         = {Deep Learning for Artificial Intelligence},
  booktitle     = {Proceedings of the IEEE International Conference on Neural Networks},
  year          = {2021},
  editor        = {Jane Smith and John Doe},
  volume        = {1},
  number = {5},
  series        = {Advances in Neural Information Processing},
  pages         = {100-120},
  address       = {Montreal, Canada},
  month         = {December},
  organization  = {IEEE},
  publisher     = {IEEE Press},
  note          = {Keynote paper on recent advancements in deep learning},
  isbn          = {978-1-5386-4637-1},
  doi           = {10.1109/ICNN.2021.9483948},
  url           = {https://ieeexplore.ieee.org/document/9483948},
  annote        = {A seminal work on how deep learning transforms AI},
  abstract      = {This paper explores cutting-edge deep learning techniques and their impact on the development of artificial intelligence...},
  keywords      = {Deep Learning, Artificial Intelligence, Neural Networks},
  language      = {English}
}


@incollection{quantum_computation,
  author        = {Michael A. Nielsen and Isaac L. Chuang},
  title         = {Quantum Computation and Quantum Information},
  booktitle     = {Handbook of Quantum Information Science},
  publisher     = {Springer},
  year          = {2026},
  editor        = {Charles H. Bennett and David P. DiVincenzo},
  volume        = {4},
  series        = {Quantum Science and Technology},
  chapter       = {10},
  pages         = {250-300},
  address       = {Berlin, Germany},
  month         = {October},
  note          = {A comprehensive overview of the fundamentals of quantum computation},
  isbn          = {978-3-540-88702-7},
  doi           = {10.1007/springerreference_303198},
  url           = {https://www.springer.com/gp/book/9783540887027},
  annote        = {Essential reading for researchers entering the field of quantum information},
  abstract      = {This chapter delves into the principles of quantum computing, offering an accessible yet thorough introduction...},
  keywords      = {Quantum Computing, Quantum Information, Computational Models},
  language      = {English},
  price         = {45.00},
  size          = {50 pages}
}


In [40]:
#RegEx zum FInden der Seiten
pageFinder = ", (?:pp\.? )?\d+-\d+,"
doi1 = "https:\/\/doi\.org"
doi2 = "DOI:"

#if havard und book:
volume = ", \d+,"
number = ", \d+,"
edition1 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) ed\."
edition2 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) edn\."

#if APA or Havard then
volume = "\(Vol\. \d+" #für Volume und number noch als Bedingung, dass es nach Titel stehen muss, also Index > Titel-Index
number = "Issue \d+"
yearApaHavard = "\(\d{4}\)"
editorRegEx = " (\()?(Eds\.|Eds|Ed|ed|Ed\.|ed\.|eds\.|editor|editors|edited by )(\))?"

#editor bei anderen


#Beachte: Editor und Edition werden oft mit gleichen Abkürzungen versehen, auch im gleichen Stil! Nur Edition wird oft
#klein geschrieben. Daher zustzlich prüfen, ob NER das als Person erkennt.

#Beachte: ieee und mla trennen mit , und nicht mit .
#if ieee and mla
volumeIeeeMla = ", vol\. \d+" 
noIeeeMla = ", no\. \d+"

etAl = "et al."

yearACM = "\. \d{4}\."

#Man sollte zunächst die Felder extrahieren, die zu 100% sicher erkennen kann wie Volume oder Number, doi, Autoren...-> 
#Titel, booktitle, series und journal danach

#man sollte eine Überdeckungsprüfung machen: Also Bspw. sagt Space im Bereich 25-50 ist eine Orga, Regex sagt, 
#dort ist ein (Eds.)
#zu finden und Huggingface sagt, von 25-45 ist was. Dann sollte der minimalteste Wert und maximalste Wert 
#genommen werden und so lange nach links und analog nach rechts gehen, bis wieder ein Punkt kommt.
# Da erkannte Felder aus dem String entfernt werden, muss der String am Ende leer sein. 
#Daher sollten zunächste die Dinge ausgeschnitten werden wo der Algo sich am sichersten ist
#TODO: Grundsätzliche Reihenfolge der BibTex-Einträge ermitteln.
#TODO: Für jede Extrahierung eines Feldes eine eigene Unterroutine schreiben? -> In der Oberschleife sollte Style sein



print(re.search(editorRegEx, "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). Chapter Title. In Johnson, E. F. & Lee, R. H. (Eds.), Book"))

<re.Match object; span=(93, 100), match=' (Eds.)'>


In [41]:
text="Hawking, S., Greene, B., Trump, M. and Soy, S. (2025). Advanced Concepts and John Miller in Theoretical Physics (3rd ed., Vol. 1). Cambridge University Press. https://doi.org/10.1017/example.book.2025"
text = "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). Chat GPT & Google Bard AI: A Review. 2023 International Conference on IoT, Communication and Automation Technology (ICICAT), 1–6. https://doi.org/10.1109/ICICAT57735.2023.10263706"
text = "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). Chapter Title. In Johnson, E. F. & Lee, R. H. (Eds.), Book Title. Publisher Name."
#text = "Singh, S. K. and Peterman, P. (2023). Chapter and Title. Trump, M. and Soy, S. Book Title. Publisher Name."
#text = "S. K. Singh, S. Kumar & P. S. Mehra, (2023). Chapter Title. In E. F. Johnson & R. H. Lee (Eds.), Book Title. Publisher Name."
#text = "Tu Anh Nguyen, Eugene Kharitonov, Jade Copet, Yossi Adi, Wei-Ning Hsu, Ali Elkahky, Paden Tomasello, Robin Algayres, Benoît Sagot, Abdelrahman Mohamed, and Emmanuel Dupoux. 2023. Generative spoken dialogue language modeling. Transactions of the Association for Computational Linguistics 11, (2023), 250–266. URL: https://aclanthology.org/2023.tacl-1.15, doi:10.1162/tacl_a_00545"
#text = "Hinton G, Bengio Y, LeCun Y (2021) Deep Learning for Artificial Intelligence. In: Smith J, Doe J (eds) Proceedings of the IEEE International Conference on Neural Networks. IEEE Press, Montreal, Canada, pp 100–120"


In [48]:
import re

def find_First_Term(text, search_terms):
    # Initialisiere mit einem hohen Wert
    min_index = float('inf')
    end_index = 0
    andTyp = ""
    
    # Suche jeden Suchbegriff in dem Text und behalte den kleinsten Index
    for term in search_terms:
        index = text.find(term)
        if index != -1 and index < min_index:
            min_index = index
            end_index = min_index + len(term) - 1
            andTyp = term
    
    # Wenn min_index unverändert ist, wurde keiner der Begriffe gefunden
    return (min_index, end_index, andTyp) if min_index != float('inf') else (-1, -1, "")

import string

def is_punctuation(s):
    allowed_chars = string.punctuation.replace('&', '') + ' '
    return all(char in allowed_chars for char in s)

ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)

#index neu setzen, da diese nicht automatich geupdates werden
df_PER = df_outputs[df_outputs["entity_group"] == "PER"].reset_index(drop=True)
df_PER_len = len(df_PER.index)


print(df_PER)

fullNameInOneWord = False
allFullNamesInOneWord = True
nameAbgekürzt = False
surenameFirst = False

for index in df_PER.index.values.tolist():
    if "." == text[df_PER["end"].iloc[index]] and len(text[df_PER["start"].iloc[index]:df_PER["end"].iloc[index] + 1]) == 2:
        nameAbgekürzt = True
for element in text:
    if element == ",":
        break
    if element in [".", " "]:
        surenameFirst = True

startIndexPER = df_PER["start"].iloc[0]
endIndexPER = df_PER["end"].iloc[-1] + 1
#Beachte, ein ., auch zwischen Autoren und Titel vorkommen kann
authors = text[startIndexPER:endIndexPER]
search_terms = []

if "et al."in text:
    search_terms = [", et al.", " et al."]
    firstStartIndexAnd, firstEndIndexAnd, andTyp = find_First_Term(text, search_terms)
    print("find_First_Term {0}, {1}, {2}".format(firstStartIndexAnd, firstEndIndexAnd, andTyp))



#prüfe, wo der letzte Autor in der Autorenkette ist, um Namen auszuschließen, die im Titel sein können
#vorher prüfen, wo et al. steht, weil so erkennt man ende sicher
#zusäzich kann man noch prüfen, wo das erst "and" oder "&" kommt




search_terms = [" and ", ", and ", " & ", ", & ", "., & ", "., and ", ". and ", ". & "]
editorRegEx = " (\()?(Eds\.|Eds|Ed|ed|Ed\.|ed\.|eds\.|editor|editors)(\))?"
index_df_PER_List = df_PER.index.values.tolist()
onlyPunctuation = False
onlyAnd = False
editedBy = False
authorsDetected = True
editorsDetected = False
startIndexAuthors = -1
endIndexAuthors = -1
startIndexEditors = -1
endIndexEditors = -1
startIndexEditorMarker = -1
endIndexEditorMarker = -1
chainStartIndex = -1
setChainStart = True

for index in index_df_PER_List:
    #beachte: Hiermit lese ich immer schon vor!
    if index < len(index_df_PER_List) - 1:
        textBetweenNames = text[df_PER["end"].iloc[index]:df_PER["start"].iloc[index + 1]]
    else:
        textBetweenNames = text[df_PER["end"].iloc[index]:]
    onlyPunctuation = is_punctuation(textBetweenNames)
    print(f'textBetweenNames: {textBetweenNames}')
    #print(f'onlyPunctuation: {onlyPunctuation}')
    firstStartIndex, firstEndIndex, andTyp = find_First_Term(textBetweenNames, search_terms)
    onlyAnd = textBetweenNames == andTyp
    #print(f'onlyAnd: {onlyAnd}')
    #print(f'nothing: {not onlyAnd and not onlyPunctuation}')
    if not onlyPunctuation and not onlyAnd:
        firstStartIndex, firstEndIndex, editorTyp = find_First_Term(textBetweenNames, ["edited by"])
        editedBy = firstEndIndex + 2 == df_PER["start"].iloc[index]
        print(f'editedBy: {editedBy}')
        if editedBy:
            startIndexEditorMarker = firstStartIndex
            endIndexEditorMarker = firstEndIndex
            editorsDetected = True
        else:
            print(f'x: {re.search(editorRegEx, text)}')
            if re.search(editorRegEx, textBetweenNames):
                x = re.search(editorRegEx, text)
                print(f'x: {x.start()}')
                if is_punctuation(text[df_PER["end"].iloc[index]:x.start()]):
                    editorsDetected = True
                    startIndexEditorMarker = x.start()
                    endIndexEditorMarker = x.end()

    
    if setChainStart: 
        chainStartIndex = df_PER["start"].iloc[index]
        #Solange das auf False, sollen der Substring erweitert werden, also start bleibt konstant
        print(f'chainStartIndex: {chainStartIndex}')
        setChainStart = False
    #Dann gab es einen Bruch in der Autorenkette. Also bin ich in einer Lücke zwischen den AUtoren
    #Dann ist Nächster Block wieder ein Autor
    if not onlyPunctuation and not onlyAnd:
        setChainStart = True
        #die Auotren kommen immer zuerst im STring
        if authorsDetected:
            startIndexAuthors = chainStartIndex
            endIndexAuthors = df_PER["end"].iloc[index]
            authorsDetected = False
        if editorsDetected:
            startIndexEditors = chainStartIndex
            endIndexEditors = df_PER["end"].iloc[index]
            break


authors = text[startIndexAuthors:endIndexAuthors]
editors = text[startIndexEditors:endIndexEditors]
print("authors: {0} ".format(authors))
print("editors: {0} ".format(editors))
print(f'Index Editor Marker: {startIndexEditorMarker}, {endIndexEditorMarker}')
andInAuthors = find_First_Term(authors, search_terms)[2]
print(andInAuthors)
andInEditors = find_First_Term(editors, search_terms)[2]
finalAuthors = ""
finalEditors = ""

if surenameFirst:
    #hier völlig egal, ob er einzelne Initialen in ein eigenes Word gesteckt hat, obwohl es noch Nachnamen gib
    search_terms = [" and ", ", and ", " & ", ", & "]
    andInAuthors = find_First_Term(authors, search_terms)[2]
    authors = authors.replace(andInAuthors, " and ")
    finalAuthors = authors.replace(", ", " and ")
elif andInAuthors != "":
    if "., " in authors:
        print("Fall ., {0}".format(authors))
        search_terms = ["., and ", "., & ", ". and ", ". & "]
        andInAuthors = find_First_Term(authors, search_terms)[2]
        authors = authors.replace(andInAuthors, "., ")
        authors = authors.split("., ")
        authors = [name + "." for name in authors]
        authors = [name.replace("..",".") for name in authors]
        for author in authors[:-1]:
            buffer = author.split(", ")
            finalAuthors = finalAuthors + buffer[1] + " " + buffer[0] + " and "
        buffer = authors[-1].split(", ")
        finalAuthors = finalAuthors + buffer[1] + " " +  buffer[0]
    elif ", " in authors:
        print("Fall , {0}".format(authors))
        search_terms = [", and ", ", & ", " and ", " & "]
        andInAuthors = find_First_Term(authors, search_terms)[2]
        authors = authors.replace(andInAuthors, ", ")
        authors = authors.split(", ")
        for i in range(0, len(authors) - 3, 2):
            finalAuthors = finalAuthors + authors[i+1] + " " + authors[i] + " and "
        finalAuthors = finalAuthors + authors[len(authors) - 1] + " " + authors[len(authors) - 2]
else:
    print("Fall else {0}".format(authors))
    authors = authors.split(", ")
    finalAuthros = authors[1] + authors[0]


text = text[startIndexMaxDistanceAuthorNames + 2:endIndexMaxDistanceAuthorNames - 1] + text[endIndexMaxDistanceAuthorNames:endIndexPER]
print(finalAuthors)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\larsl\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

   entity_group     score     word  start  end
0           PER  0.999411    Singh      0    5
1           PER  0.996913        S      7    8
2           PER  0.997643        K     10   11
3           PER  0.999555    Kumar     14   19
4           PER  0.998111        S     21   22
5           PER  0.987601    Mehra     27   32
6           PER  0.998999        P     34   35
7           PER  0.998115        S     37   38
8           PER  0.999267  Johnson     66   73
9           PER  0.990239        E     75   76
10          PER  0.965848        F     78   79
11          PER  0.998798      Lee     83   86
12          PER  0.997356        R     88   89
13          PER  0.988298        H     91   92
textBetweenNames: , 
chainStartIndex: 0
textBetweenNames: . 
textBetweenNames: ., 
textBetweenNames: , 
textBetweenNames: ., & 
textBetweenNames: , 
textBetweenNames: . 
textBetweenNames: . (2023). Chapter Title. In 
editedBy: False
x: <re.Match object; span=(93, 100), match=' (Eds.)'>
textBetw

NameError: name 'startIndexMaxDistanceAuthorNames' is not defined

In [11]:
import string

s = "&! "
print(all(char in string.punctuation for char in s))

False


In [34]:
def find_First_Term(text, search_terms):
    # Initialisiere mit einem hohen Wert
    min_index = float('inf')
    end_index = 0
    andTyp = ""
    
    # Suche jeden Suchbegriff in dem Text und behalte den kleinsten Index
    for term in search_terms:
        index = text.find(term)
        if index != -1 and index < min_index:
            min_index = index
            end_index = min_index + len(term) - 1
            andTyp = term
    
    # Wenn min_index unverändert ist, wurde keiner der Begriffe gefunden
    return (min_index, end_index, andTyp) if min_index != float('inf') else (-1, -1, "")

import string

def is_punctuation(s):
    allowed_chars = string.punctuation.replace('&', '') + ' '
    return all(char in allowed_chars for char in s)

[False, False, False, False, True, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False]
[False, False, False, False, True, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, ' and ', ', and ', ' & ', ', & ', '., & ', '., and ']


Idee: Nun zunächst Jahr, Volume, Seiten, Edition und URL/DOI extrahieren. Den Rest (also Titel, Publisher, Series), dann nochmal den SpacyParer drüber laufen lassen, weil 

In [16]:
s = "This Book was edited by Martin Trump" 
print(find_First_Term(s, ["edited by"]))
ner_tagger = pipeline("ner", aggregation_strategy="simple")
text = s
outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)
print(df_outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


(14, 22, 'edited by')


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  entity_group     score          word  start  end
0         MISC  0.871492          Book      5    9
1          PER  0.999499  Martin Trump     24   36


In [ ]:
#Test, um Datum Zuverlässig erkannt wird

import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Hinton, G., Bengio, Y. and LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.9483948")
for ent in doc.ents:
  print(ent, ent.start_char-ent.sent.start_char, ent.end_char-ent.sent.start_char, ent.label_)

In [50]:
text = "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). Chapter Title. In Johnson, E. F. & Lee, R. H. (Eds.), Book Title. Publisher Name."
print(text[92])

.
